In [86]:
pip install requests beautifulsoup4 markdownify

^C
Fatal Python error: init_import_size: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site.py", line 73, in <module>
    import os
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/os.py", line 59, in <module>
    import posixpath as path
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 844, in exec_module
  File "<frozen importlib._bootstrap_external>", line 939, in get_code
  File "<frozen importlib._bootstrap_external>", line 1037, in get_data
KeyboardInterrupt
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

import requests
from bs4 import BeautifulSoup as bs
import markdownify
import time

blog_url = 'https://hyostech.tistory.com'
image_save_dir = 'assets/images/posts'

for page_no in range(5, 148 + 1):
    print(page_no, ' ... ', end='')
    html = requests.get(f'{blog_url}/{page_no}')

    if html.status_code == 404:
        print('404')
        continue

    soup = bs(html.text, "html.parser")

    title = soup.select_one(".article-header .title-article")
    print(title.text, ' ... ', end='')

    content = soup.select_one(".article-view .contents_style")
    content_md = markdownify.markdownify(str(content), heading_style="ATX")
    # print(content_md[:10], ' ... ')
    # print(content_md)

    category = soup.select_one(".article-header .category")
    # print(category.text)
    
    date = soup.select_one(".article-header .date")
    year, month, day, _ = date.text.split('.')
    # print(year, month, day, hour, minute)

    tags = soup.select(".article-tag .box-tag a")
    tags = '\n  '.join(list(map(lambda tag: f'- {tag.text}', tags)))

    md_base_name = f'{year.strip()}-{month.strip()}-{day.strip()}-tistory-post-{page_no}'
    image_dir = os.path.join(image_save_dir, md_base_name)

    if content.select("img"):
        os.makedirs(image_dir, exist_ok=True)
    for n, image in enumerate(content.select("img"), start=1):
        src = image['src']
        # print(src)
        image_path = os.path.join(image_dir, f'img-{n}.png')
        open(image_path, 'wb').write(requests.get(src).content)
        content_md = content_md.replace(src, f'/{image_path}')

    metadata = f"""---
title: {title.text}
toc: true
categories:
  - {category.text}
tags:
  {tags}
---

"""
    content_md = metadata + content_md
    open(os.path.join('_posts', f'{md_base_name}.md'), 'w').write(content_md)

    print('OK')